In [1]:
import netCDF4 as nc
import numpy as np
from matplotlib import pyplot as plt
import os
import json
import pandas as pd
import seaborn as sns
import xarray as xr

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
!pwd

/work/scripts/caribou-poker-exp


In [4]:
os.environ['HDF5_USE_FILE_LOCKING']='FALSE'

In [5]:
#establish site coordinates

run_name='BONA-birch'

station_lat = 65.15401 #caribou creek
station_lon = -147.50258 #caribou creek

#get netcdf coordinates from runmask
#runmask = nc.Dataset('/data/workflows/BONA-birch/run-mask.nc') #original climate dataset
runmask = nc.Dataset('/data/input-catalog/cpcrw_towers_downscaled/run-mask.nc') #downscaled climate dataset
lats=runmask.variables['lat'][:]
lons=runmask.variables['lon'][:]

#get distance between station and each cell
ydist = lats-station_lat
xdist = lons-station_lon
euc_dist = (ydist**2 + xdist**2)**.5

y_x = np.unravel_index(np.argmin(euc_dist),np.shape(lats))
print('y coordinate: {}'.format(y_x[0]))
print('x coordinate: {}'.format(y_x[1]))

y coordinate: 0
x coordinate: 0


# BONA Birch

In [6]:
!rm -r /data/workflows/BONA-birch/

In [7]:
%cd /work

/work


In [8]:
#set working directory, original input
#!scripts/setup_working_directory.py \
#--input-data-path /data/input-catalog/caribou-poker_merged/ \
#/data/workflows/BONA-birch/

In [9]:
#set working directory, downscaled input
!scripts/setup_working_directory.py \
--input-data-path /data/input-catalog/cpcrw_towers_downscaled/ \
/data/workflows/BONA-birch/

In [10]:
# Adjust the config file
CONFIG_FILE = os.path.join('/data/workflows/BONA-birch/', 'config/config.js')
# Read the existing data into memory
with open(CONFIG_FILE, 'r') as f:
    config = json.load(f)
    
    config['IO']['output_nc_eq'] = 1 # Modify value...

# Write it back..
with open(CONFIG_FILE, 'w') as f:
    json.dump(config, f, indent=2)

In [11]:
#poker flats: 0, 1
#caribou creek: 3, 0 # original climate
#caribou creek: 0, 0, #downscaled climate
# setup runmask
!runmask-util.py --reset \
--yx 0 0 \
--show \
/data/workflows/BONA-birch/run-mask.nc

========== BEFORE ==================================
** Keep in mind that in this display the origin is the upper 
** left of the grid! This is opposite of the way that ncdump 
** and ncview display data (origin is lower left)!!

'/data/workflows/BONA-birch/run-mask.nc'
<class 'netCDF4._netCDF4.Variable'>
int64 run(Y, X)
    _FillValue: -999
    standard_name: mask
    units: 
    grid_mapping: albers_conical_equal_area
unlimited dimensions: 
current shape = (1, 1)
filling on
[[-147]]

========== AFTER ==================================
** Keep in mind that in this display the origin is the upper 
** left of the grid! This is opposite of the way that ncdump 
** and ncview display data (origin is lower left)!!

'/data/workflows/BONA-birch/run-mask.nc'
<class 'netCDF4._netCDF4.Variable'>
int64 run(Y, X)
    _FillValue: -999
    standard_name: mask
    units: 
    grid_mapping: albers_conical_equal_area
unlimited dimensions: 
current shape = (1, 1)
filling on
[[1]]



In [12]:
path_to_soil_input='/data/input-catalog/caribou-poker_merged/soil-texture.nc'
soil_dataset = nc.Dataset(path_to_soil_input)
print(soil_dataset)
print('target cell is {}% clay, {}% sand, and {}% silt'.format(soil_dataset['pct_clay'][y_x[0], y_x[1]], 
                                                               soil_dataset['pct_sand'][y_x[0], y_x[1]], 
                                                               soil_dataset['pct_silt'][y_x[0], y_x[1]]))
soil_dataset.close()

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    Conventions: CF-1.5
    source: /home/UA/tcarman2/dvm-dos-tem/./scripts/create_region_input.py::fill_soil_texture_file --xoff 221779.360241855 --yoff 1638167.29849358
    dimensions(sizes): Y(4), X(2)
    variables(dimensions): |S1 albers_conical_equal_area(), float32 lat(Y, X), float32 lon(Y, X), float32 pct_clay(Y, X), float32 pct_sand(Y, X), float32 pct_silt(Y, X), int32 x(X), int32 y(Y)
    groups: 
target cell is 8.81557846069336% clay, 42.533843994140625% sand, and 48.650577545166016% silt


In [13]:
#soil_dataset = nc.Dataset(path_to_soil_input, 'r+')
#soil_dataset['pct_clay'][y_x[0], y_x[1]] = 10 # originally 8.81557846069336
#soil_dataset['pct_sand'][y_x[0], y_x[1]] = 27 # originally 42.533843994140625
#soil_dataset['pct_silt'][y_x[0], y_x[1]] = 63 # origially 48.650577545166016
#print('target cell is {}% clay, {}% sand, and {}% silt'.format(soil_dataset['pct_clay'][y_x[0], y_x[1]], 
#                                                               soil_dataset['pct_sand'][y_x[0], y_x[1]], 
#                                                               soil_dataset['pct_silt'][y_x[0], y_x[1]]))
#drainage.close()


In [14]:
!scripts/outspec_utils.py ../data/workflows/BONA-birch/config/output_spec.csv --empty
!scripts/outspec_utils.py ../data/workflows/BONA-birch/config/output_spec.csv --on CMTNUM yearly
!scripts/outspec_utils.py ../data/workflows/BONA-birch/config/output_spec.csv --on GPP monthly
!scripts/outspec_utils.py ../data/workflows/BONA-birch/config/output_spec.csv --on RG monthly compartment
!scripts/outspec_utils.py ../data/workflows/BONA-birch/config/output_spec.csv --on RH monthly layer
!scripts/outspec_utils.py ../data/workflows/BONA-birch/config/output_spec.csv --on RM monthly compartment
!scripts/outspec_utils.py ../data/workflows/BONA-birch/config/output_spec.csv --on NPP monthly
!scripts/outspec_utils.py ../data/workflows/BONA-birch/config/output_spec.csv --on ALD yearly 
!scripts/outspec_utils.py ../data/workflows/BONA-birch/config/output_spec.csv --on SHLWC yearly monthly
!scripts/outspec_utils.py ../data/workflows/BONA-birch/config/output_spec.csv --on DEEPC yearly
!scripts/outspec_utils.py ../data/workflows/BONA-birch/config/output_spec.csv --on MINEC yearly
!scripts/outspec_utils.py ../data/workflows/BONA-birch/config/output_spec.csv --on ORGN yearly
!scripts/outspec_utils.py ../data/workflows/BONA-birch/config/output_spec.csv --on AVLN yearly
!scripts/outspec_utils.py ../data/workflows/BONA-birch/config/output_spec.csv --on LTRFALC monthly
!scripts/outspec_utils.py ../data/workflows/BONA-birch/config/output_spec.csv --on LWCLAYER monthly
!scripts/outspec_utils.py ../data/workflows/BONA-birch/config/output_spec.csv --on TLAYER monthly
!scripts/outspec_utils.py ../data/workflows/BONA-birch/config/output_spec.csv --on LAYERDEPTH monthly
!scripts/outspec_utils.py ../data/workflows/BONA-birch/config/output_spec.csv --on LAYERDZ monthly
!scripts/outspec_utils.py ../data/workflows/BONA-birch/config/output_spec.csv --on EET monthly
!scripts/outspec_utils.py ../data/workflows/BONA-birch/config/output_spec.csv --on TRANSPIRATION monthly PFT
!scripts/outspec_utils.py ../data/workflows/BONA-birch/config/output_spec.csv --on LAI monthly PFT
!scripts/outspec_utils.py ../data/workflows/BONA-birch/config/output_spec.csv --on VEGC monthly PFT compartment
!scripts/outspec_utils.py ../data/workflows/BONA-birch/config/output_spec.csv --on BURNVEG2AIRC monthly

In [15]:
%cd /data/workflows/BONA-birch

/data/workflows/BONA-birch


In [16]:
!dvmdostem --force-cmt=14 --log-level='err' --eq-yrs=1000 --sp-yrs=300 --tr-yrs=122 --sc-yrs=0

Setting up logging...
[err] [PRE-RUN->Y] y: 0 x: 0 Year: 0
[err] [PRE-RUN->Y] y: 0 x: 0 Year: 1
[err] [PRE-RUN->Y] y: 0 x: 0 Year: 2
[err] [PRE-RUN->Y] y: 0 x: 0 Year: 3
[err] [PRE-RUN->Y] y: 0 x: 0 Year: 4
[err] [PRE-RUN->Y] y: 0 x: 0 Year: 5
[err] [PRE-RUN->Y] y: 0 x: 0 Year: 6
[err] [PRE-RUN->Y] y: 0 x: 0 Year: 7
[err] [PRE-RUN->Y] y: 0 x: 0 Year: 8
[err] [PRE-RUN->Y] y: 0 x: 0 Year: 9
[fatal] [EQ] Equilibrium Initial Year Count: 1000
[err] [EQ] The model will not run enough years to complete a disturbance cycle!
[fatal] [EQ] Running Equilibrium, 1000 years.
[err] [EQ->Y] y: 0 x: 0 Year: 0
[err] [EQ->Y] y: 0 x: 0 Year: 1
[err] [EQ->Y] y: 0 x: 0 Year: 2
[err] [EQ->Y] y: 0 x: 0 Year: 3
[err] [EQ->Y] y: 0 x: 0 Year: 4
[err] [EQ->Y] y: 0 x: 0 Year: 5
[err] [EQ->Y] y: 0 x: 0 Year: 6
[err] [EQ->Y] y: 0 x: 0 Year: 7
[err] [EQ->Y] y: 0 x: 0 Year: 8
[err] [EQ->Y] y: 0 x: 0 Year: 9
[err] [EQ->Y] y: 0 x: 0 Year: 10
[err] [EQ->Y] y: 0 x: 0 Year: 11
[err] [EQ->Y] y: 0 x: 0 Year: 12
[err] [EQ->Y]

[err] [EQ->Y] y: 0 x: 0 Year: 228
[err] [EQ->Y] y: 0 x: 0 Year: 229
[err] [EQ->Y] y: 0 x: 0 Year: 230
[err] [EQ->Y] y: 0 x: 0 Year: 231
[err] [EQ->Y] y: 0 x: 0 Year: 232
[err] [EQ->Y] y: 0 x: 0 Year: 233
[err] [EQ->Y] y: 0 x: 0 Year: 234
[err] [EQ->Y] y: 0 x: 0 Year: 235
[err] [EQ->Y] y: 0 x: 0 Year: 236
[err] [EQ->Y] y: 0 x: 0 Year: 237
[err] [EQ->Y] y: 0 x: 0 Year: 238
[err] [EQ->Y] y: 0 x: 0 Year: 239
[err] [EQ->Y] y: 0 x: 0 Year: 240
[err] [EQ->Y] y: 0 x: 0 Year: 241
[err] [EQ->Y] y: 0 x: 0 Year: 242
[err] [EQ->Y] y: 0 x: 0 Year: 243
[err] [EQ->Y] y: 0 x: 0 Year: 244
[err] [EQ->Y] y: 0 x: 0 Year: 245
[err] [EQ->Y] y: 0 x: 0 Year: 246
[err] [EQ->Y] y: 0 x: 0 Year: 247
[err] [EQ->Y] y: 0 x: 0 Year: 248
[err] [EQ->Y] y: 0 x: 0 Year: 249
[err] [EQ->Y] y: 0 x: 0 Year: 250
[err] [EQ->Y] y: 0 x: 0 Year: 251
[err] [EQ->Y] y: 0 x: 0 Year: 252
[err] [EQ->Y] y: 0 x: 0 Year: 253
[err] [EQ->Y] y: 0 x: 0 Year: 254
[err] [EQ->Y] y: 0 x: 0 Year: 255
[err] [EQ->Y] y: 0 x: 0 Year: 256
[err] [EQ->Y] 

[err] [EQ->Y] y: 0 x: 0 Year: 469
[err] [EQ->Y] y: 0 x: 0 Year: 470
[err] [EQ->Y] y: 0 x: 0 Year: 471
[err] [EQ->Y] y: 0 x: 0 Year: 472
[err] [EQ->Y] y: 0 x: 0 Year: 473
[err] [EQ->Y] y: 0 x: 0 Year: 474
[err] [EQ->Y] y: 0 x: 0 Year: 475
[err] [EQ->Y] y: 0 x: 0 Year: 476
[err] [EQ->Y] y: 0 x: 0 Year: 477
[err] [EQ->Y] y: 0 x: 0 Year: 478
[err] [EQ->Y] y: 0 x: 0 Year: 479
[err] [EQ->Y] y: 0 x: 0 Year: 480
[err] [EQ->Y] y: 0 x: 0 Year: 481
[err] [EQ->Y] y: 0 x: 0 Year: 482
[err] [EQ->Y] y: 0 x: 0 Year: 483
[err] [EQ->Y] y: 0 x: 0 Year: 484
[err] [EQ->Y] y: 0 x: 0 Year: 485
[err] [EQ->Y] y: 0 x: 0 Year: 486
[err] [EQ->Y] y: 0 x: 0 Year: 487
[err] [EQ->Y] y: 0 x: 0 Year: 488
[err] [EQ->Y] y: 0 x: 0 Year: 489
[err] [EQ->Y] y: 0 x: 0 Year: 490
[err] [EQ->Y] y: 0 x: 0 Year: 491
[err] [EQ->Y] y: 0 x: 0 Year: 492
[err] [EQ->Y] y: 0 x: 0 Year: 493
[err] [EQ->Y] y: 0 x: 0 Year: 494
[err] [EQ->Y] y: 0 x: 0 Year: 495
[err] [EQ->Y] y: 0 x: 0 Year: 496
[err] [EQ->Y] y: 0 x: 0 Year: 497
[err] [EQ->Y] 

[err] [EQ->Y] y: 0 x: 0 Year: 710
[err] [EQ->Y] y: 0 x: 0 Year: 711
[err] [EQ->Y] y: 0 x: 0 Year: 712
[err] [EQ->Y] y: 0 x: 0 Year: 713
[err] [EQ->Y] y: 0 x: 0 Year: 714
[err] [EQ->Y] y: 0 x: 0 Year: 715
[err] [EQ->Y] y: 0 x: 0 Year: 716
[err] [EQ->Y] y: 0 x: 0 Year: 717
[err] [EQ->Y] y: 0 x: 0 Year: 718
[err] [EQ->Y] y: 0 x: 0 Year: 719
[err] [EQ->Y] y: 0 x: 0 Year: 720
[err] [EQ->Y] y: 0 x: 0 Year: 721
[err] [EQ->Y] y: 0 x: 0 Year: 722
[err] [EQ->Y] y: 0 x: 0 Year: 723
[err] [EQ->Y] y: 0 x: 0 Year: 724
[err] [EQ->Y] y: 0 x: 0 Year: 725
[err] [EQ->Y] y: 0 x: 0 Year: 726
[err] [EQ->Y] y: 0 x: 0 Year: 727
[err] [EQ->Y] y: 0 x: 0 Year: 728
[err] [EQ->Y] y: 0 x: 0 Year: 729
[err] [EQ->Y] y: 0 x: 0 Year: 730
[err] [EQ->Y] y: 0 x: 0 Year: 731
[err] [EQ->Y] y: 0 x: 0 Year: 732
[err] [EQ->Y] y: 0 x: 0 Year: 733
[err] [EQ->Y] y: 0 x: 0 Year: 734
[err] [EQ->Y] y: 0 x: 0 Year: 735
[err] [EQ->Y] y: 0 x: 0 Year: 736
[err] [EQ->Y] y: 0 x: 0 Year: 737
[err] [EQ->Y] y: 0 x: 0 Year: 738
[err] [EQ->Y] 

[err] [EQ->Y] y: 0 x: 0 Year: 951
[err] [EQ->Y] y: 0 x: 0 Year: 952
[err] [EQ->Y] y: 0 x: 0 Year: 953
[err] [EQ->Y] y: 0 x: 0 Year: 954
[err] [EQ->Y] y: 0 x: 0 Year: 955
[err] [EQ->Y] y: 0 x: 0 Year: 956
[err] [EQ->Y] y: 0 x: 0 Year: 957
[err] [EQ->Y] y: 0 x: 0 Year: 958
[err] [EQ->Y] y: 0 x: 0 Year: 959
[err] [EQ->Y] y: 0 x: 0 Year: 960
[err] [EQ->Y] y: 0 x: 0 Year: 961
[err] [EQ->Y] y: 0 x: 0 Year: 962
[err] [EQ->Y] y: 0 x: 0 Year: 963
[err] [EQ->Y] y: 0 x: 0 Year: 964
[err] [EQ->Y] y: 0 x: 0 Year: 965
[err] [EQ->Y] y: 0 x: 0 Year: 966
[err] [EQ->Y] y: 0 x: 0 Year: 967
[err] [EQ->Y] y: 0 x: 0 Year: 968
[err] [EQ->Y] y: 0 x: 0 Year: 969
[err] [EQ->Y] y: 0 x: 0 Year: 970
[err] [EQ->Y] y: 0 x: 0 Year: 971
[err] [EQ->Y] y: 0 x: 0 Year: 972
[err] [EQ->Y] y: 0 x: 0 Year: 973
[err] [EQ->Y] y: 0 x: 0 Year: 974
[err] [EQ->Y] y: 0 x: 0 Year: 975
[err] [EQ->Y] y: 0 x: 0 Year: 976
[err] [EQ->Y] y: 0 x: 0 Year: 977
[err] [EQ->Y] y: 0 x: 0 Year: 978
[err] [EQ->Y] y: 0 x: 0 Year: 979
[err] [EQ->Y] 

[err] [SP->Y] y: 0 x: 0 Year: 190
[err] [SP->Y] y: 0 x: 0 Year: 191
[err] [SP->Y] y: 0 x: 0 Year: 192
[err] [SP->Y] y: 0 x: 0 Year: 193
[err] [SP->Y] y: 0 x: 0 Year: 194
[err] [SP->Y] y: 0 x: 0 Year: 195
[err] [SP->Y] y: 0 x: 0 Year: 196
[err] [SP->Y] y: 0 x: 0 Year: 197
[err] [SP->Y] y: 0 x: 0 Year: 198
[err] [SP->Y] y: 0 x: 0 Year: 199
[err] [SP->Y] y: 0 x: 0 Year: 200
[err] [SP->Y] y: 0 x: 0 Year: 201
[err] [SP->Y] y: 0 x: 0 Year: 202
[err] [SP->Y] y: 0 x: 0 Year: 203
[err] [SP->Y] y: 0 x: 0 Year: 204
[err] [SP->Y] y: 0 x: 0 Year: 205
[err] [SP->Y] y: 0 x: 0 Year: 206
[err] [SP->Y] y: 0 x: 0 Year: 207
[err] [SP->Y] y: 0 x: 0 Year: 208
[err] [SP->Y] y: 0 x: 0 Year: 209
[err] [SP->Y] y: 0 x: 0 Year: 210
[err] [SP->Y] y: 0 x: 0 Year: 211
[err] [SP->Y] y: 0 x: 0 Year: 212
[err] [SP->Y] y: 0 x: 0 Year: 213
[err] [SP->Y] y: 0 x: 0 Year: 214
[err] [SP->Y] y: 0 x: 0 Year: 215
[err] [SP->Y] y: 0 x: 0 Year: 216
[err] [SP->Y] y: 0 x: 0 Year: 217
[err] [SP->Y] y: 0 x: 0 Year: 218
[err] [SP->Y] 

In [17]:
!ls /data/workflows/BONA-birch/output/

ALD_yearly_eq.nc	    MINEC_yearly_eq.nc
ALD_yearly_sp.nc	    MINEC_yearly_sp.nc
ALD_yearly_tr.nc	    MINEC_yearly_tr.nc
AVLN_yearly_eq.nc	    NPP_monthly_eq.nc
AVLN_yearly_sp.nc	    NPP_monthly_sp.nc
AVLN_yearly_tr.nc	    NPP_monthly_tr.nc
BURNVEG2AIRC_monthly_eq.nc  ORGN_yearly_eq.nc
BURNVEG2AIRC_monthly_sp.nc  ORGN_yearly_sp.nc
BURNVEG2AIRC_monthly_tr.nc  ORGN_yearly_tr.nc
CMTNUM_yearly_eq.nc	    RG_monthly_eq.nc
CMTNUM_yearly_sp.nc	    RG_monthly_sp.nc
CMTNUM_yearly_tr.nc	    RG_monthly_tr.nc
DEEPC_yearly_eq.nc	    RH_monthly_eq.nc
DEEPC_yearly_sp.nc	    RH_monthly_sp.nc
DEEPC_yearly_tr.nc	    RH_monthly_tr.nc
EET_monthly_eq.nc	    RM_monthly_eq.nc
EET_monthly_sp.nc	    RM_monthly_sp.nc
EET_monthly_tr.nc	    RM_monthly_tr.nc
GPP_monthly_eq.nc	    SHLWC_monthly_eq.nc
GPP_monthly_sp.nc	    SHLWC_monthly_sp.nc
GPP_monthly_tr.nc	    SHLWC_monthly_tr.nc
LAI_monthly_eq.nc	    TLAYER_monthly_eq.nc
LAI_monthly_sp.nc	    TLAYER_monthly_sp.nc
LAI_monthly_tr.nc	    TLAYER_monthly_tr.nc
LAYERDE

# BONA Black Spruce

In [18]:
%cd /work

/work


In [19]:
!rm -r /data/workflows/BONA-black-spruce/

In [20]:
#set working directory
#!scripts/setup_working_directory.py \
#--input-data-path /data/input-catalog/caribou-poker_merged/ \
#/data/workflows/BONA-black-spruce/

In [21]:
#set working directory, downscaled input
!scripts/setup_working_directory.py \
--input-data-path /data/input-catalog/cpcrw_towers_downscaled/ \
/data/workflows/BONA-black-spruce/

In [22]:
# Adjust the config file
CONFIG_FILE = os.path.join('/data/workflows/BONA-black-spruce/', 'config/config.js')
# Read the existing data into memory
with open(CONFIG_FILE, 'r') as f:
    config = json.load(f)
    
    config['IO']['output_nc_eq'] = 1 # Modify value...

# Write it back..
with open(CONFIG_FILE, 'w') as f:
    json.dump(config, f, indent=2)

In [23]:
#poker flats: 0, 1
#caribou creek: 3, 0
#caribou creek downscaled: 0, 0
# setup runmask
!runmask-util.py --reset \
--yx 0 0 \
--show \
/data/workflows/BONA-black-spruce/run-mask.nc

========== BEFORE ==================================
** Keep in mind that in this display the origin is the upper 
** left of the grid! This is opposite of the way that ncdump 
** and ncview display data (origin is lower left)!!

'/data/workflows/BONA-black-spruce/run-mask.nc'
<class 'netCDF4._netCDF4.Variable'>
int64 run(Y, X)
    _FillValue: -999
    standard_name: mask
    units: 
    grid_mapping: albers_conical_equal_area
unlimited dimensions: 
current shape = (1, 1)
filling on
[[-147]]

========== AFTER ==================================
** Keep in mind that in this display the origin is the upper 
** left of the grid! This is opposite of the way that ncdump 
** and ncview display data (origin is lower left)!!

'/data/workflows/BONA-black-spruce/run-mask.nc'
<class 'netCDF4._netCDF4.Variable'>
int64 run(Y, X)
    _FillValue: -999
    standard_name: mask
    units: 
    grid_mapping: albers_conical_equal_area
unlimited dimensions: 
current shape = (1, 1)
filling on
[[1]]



In [24]:
!scripts/outspec_utils.py ../data/workflows/BONA-black-spruce/config/output_spec.csv --empty
!scripts/outspec_utils.py ../data/workflows/BONA-black-spruce/config/output_spec.csv --on CMTNUM yearly
!scripts/outspec_utils.py ../data/workflows/BONA-black-spruce/config/output_spec.csv --on GPP monthly
!scripts/outspec_utils.py ../data/workflows/BONA-black-spruce/config/output_spec.csv --on RG monthly compartment
!scripts/outspec_utils.py ../data/workflows/BONA-black-spruce/config/output_spec.csv --on RH monthly layer
!scripts/outspec_utils.py ../data/workflows/BONA-black-spruce/config/output_spec.csv --on RM monthly compartment
!scripts/outspec_utils.py ../data/workflows/BONA-black-spruce/config/output_spec.csv --on NPP monthly
!scripts/outspec_utils.py ../data/workflows/BONA-black-spruce/config/output_spec.csv --on ALD yearly
!scripts/outspec_utils.py ../data/workflows/BONA-black-spruce/config/output_spec.csv --on SHLWC yearly monthly
!scripts/outspec_utils.py ../data/workflows/BONA-black-spruce/config/output_spec.csv --on DEEPC yearly
!scripts/outspec_utils.py ../data/workflows/BONA-black-spruce/config/output_spec.csv --on MINEC yearly
!scripts/outspec_utils.py ../data/workflows/BONA-black-spruce/config/output_spec.csv --on ORGN yearly
!scripts/outspec_utils.py ../data/workflows/BONA-black-spruce/config/output_spec.csv --on AVLN yearly
!scripts/outspec_utils.py ../data/workflows/BONA-black-spruce/config/output_spec.csv --on LTRFALC monthly
!scripts/outspec_utils.py ../data/workflows/BONA-black-spruce/config/output_spec.csv --on LWCLAYER monthly
!scripts/outspec_utils.py ../data/workflows/BONA-black-spruce/config/output_spec.csv --on TLAYER monthly
!scripts/outspec_utils.py ../data/workflows/BONA-black-spruce/config/output_spec.csv --on LAYERDEPTH monthly
!scripts/outspec_utils.py ../data/workflows/BONA-black-spruce/config/output_spec.csv --on LAYERDZ monthly
!scripts/outspec_utils.py ../data/workflows/BONA-black-spruce/config/output_spec.csv --on EET monthly
!scripts/outspec_utils.py ../data/workflows/BONA-black-spruce/config/output_spec.csv --on TRANSPIRATION monthly PFT
!scripts/outspec_utils.py ../data/workflows/BONA-black-spruce/config/output_spec.csv --on LAI monthly PFT
!scripts/outspec_utils.py ../data/workflows/BONA-black-spruce/config/output_spec.csv --on VEGC monthly PFT compartment
!scripts/outspec_utils.py ../data/workflows/BONA-black-spruce/config/output_spec.csv --on BURNVEG2AIRC monthly

In [25]:
%cd /data/workflows/BONA-black-spruce

/data/workflows/BONA-black-spruce


In [26]:
#!dvmdostem --force-cmt=15 --log-level='err' --eq-yrs=1000 --sp-yrs=300 --tr-yrs=115 --sc-yrs=10
!dvmdostem --force-cmt=15 --log-level='err' --eq-yrs=1000 --sp-yrs=300 --tr-yrs=122 --sc-yrs=0

Setting up logging...
[err] [PRE-RUN->Y] y: 0 x: 0 Year: 0
[err] [PRE-RUN->Y] y: 0 x: 0 Year: 1
[err] [PRE-RUN->Y] y: 0 x: 0 Year: 2
[err] [PRE-RUN->Y] y: 0 x: 0 Year: 3
[err] [PRE-RUN->Y] y: 0 x: 0 Year: 4
[err] [PRE-RUN->Y] y: 0 x: 0 Year: 5
[err] [PRE-RUN->Y] y: 0 x: 0 Year: 6
[err] [PRE-RUN->Y] y: 0 x: 0 Year: 7
[err] [PRE-RUN->Y] y: 0 x: 0 Year: 8
[err] [PRE-RUN->Y] y: 0 x: 0 Year: 9
[fatal] [EQ] Equilibrium Initial Year Count: 1000
[err] [EQ] The model will not run enough years to complete a disturbance cycle!
[fatal] [EQ] Running Equilibrium, 1000 years.
[err] [EQ->Y] y: 0 x: 0 Year: 0
[err] [EQ->Y] y: 0 x: 0 Year: 1
[err] [EQ->Y] y: 0 x: 0 Year: 2
[err] [EQ->Y] y: 0 x: 0 Year: 3
[err] [EQ->Y] y: 0 x: 0 Year: 4
[err] [EQ->Y] y: 0 x: 0 Year: 5
[err] [EQ->Y] y: 0 x: 0 Year: 6
[err] [EQ->Y] y: 0 x: 0 Year: 7
[err] [EQ->Y] y: 0 x: 0 Year: 8
[err] [EQ->Y] y: 0 x: 0 Year: 9
[err] [EQ->Y] y: 0 x: 0 Year: 10
[err] [EQ->Y] y: 0 x: 0 Year: 11
[err] [EQ->Y] y: 0 x: 0 Year: 12
[err] [EQ->Y]

[err] [EQ->Y] y: 0 x: 0 Year: 228
[err] [EQ->Y] y: 0 x: 0 Year: 229
[err] [EQ->Y] y: 0 x: 0 Year: 230
[err] [EQ->Y] y: 0 x: 0 Year: 231
[err] [EQ->Y] y: 0 x: 0 Year: 232
[err] [EQ->Y] y: 0 x: 0 Year: 233
[err] [EQ->Y] y: 0 x: 0 Year: 234
[err] [EQ->Y] y: 0 x: 0 Year: 235
[err] [EQ->Y] y: 0 x: 0 Year: 236
[err] [EQ->Y] y: 0 x: 0 Year: 237
[err] [EQ->Y] y: 0 x: 0 Year: 238
[err] [EQ->Y] y: 0 x: 0 Year: 239
[err] [EQ->Y] y: 0 x: 0 Year: 240
[err] [EQ->Y] y: 0 x: 0 Year: 241
[err] [EQ->Y] y: 0 x: 0 Year: 242
[err] [EQ->Y] y: 0 x: 0 Year: 243
[err] [EQ->Y] y: 0 x: 0 Year: 244
[err] [EQ->Y] y: 0 x: 0 Year: 245
[err] [EQ->Y] y: 0 x: 0 Year: 246
[err] [EQ->Y] y: 0 x: 0 Year: 247
[err] [EQ->Y] y: 0 x: 0 Year: 248
[err] [EQ->Y] y: 0 x: 0 Year: 249
[err] [EQ->Y] y: 0 x: 0 Year: 250
[err] [EQ->Y] y: 0 x: 0 Year: 251
[err] [EQ->Y] y: 0 x: 0 Year: 252
[err] [EQ->Y] y: 0 x: 0 Year: 253
[err] [EQ->Y] y: 0 x: 0 Year: 254
[err] [EQ->Y] y: 0 x: 0 Year: 255
[err] [EQ->Y] y: 0 x: 0 Year: 256
[err] [EQ->Y] 

[err] [EQ->Y] y: 0 x: 0 Year: 469
[err] [EQ->Y] y: 0 x: 0 Year: 470
[err] [EQ->Y] y: 0 x: 0 Year: 471
[err] [EQ->Y] y: 0 x: 0 Year: 472
[err] [EQ->Y] y: 0 x: 0 Year: 473
[err] [EQ->Y] y: 0 x: 0 Year: 474
[err] [EQ->Y] y: 0 x: 0 Year: 475
[err] [EQ->Y] y: 0 x: 0 Year: 476
[err] [EQ->Y] y: 0 x: 0 Year: 477
[err] [EQ->Y] y: 0 x: 0 Year: 478
[err] [EQ->Y] y: 0 x: 0 Year: 479
[err] [EQ->Y] y: 0 x: 0 Year: 480
[err] [EQ->Y] y: 0 x: 0 Year: 481
[err] [EQ->Y] y: 0 x: 0 Year: 482
[err] [EQ->Y] y: 0 x: 0 Year: 483
[err] [EQ->Y] y: 0 x: 0 Year: 484
[err] [EQ->Y] y: 0 x: 0 Year: 485
[err] [EQ->Y] y: 0 x: 0 Year: 486
[err] [EQ->Y] y: 0 x: 0 Year: 487
[err] [EQ->Y] y: 0 x: 0 Year: 488
[err] [EQ->Y] y: 0 x: 0 Year: 489
[err] [EQ->Y] y: 0 x: 0 Year: 490
[err] [EQ->Y] y: 0 x: 0 Year: 491
[err] [EQ->Y] y: 0 x: 0 Year: 492
[err] [EQ->Y] y: 0 x: 0 Year: 493
[err] [EQ->Y] y: 0 x: 0 Year: 494
[err] [EQ->Y] y: 0 x: 0 Year: 495
[err] [EQ->Y] y: 0 x: 0 Year: 496
[err] [EQ->Y] y: 0 x: 0 Year: 497
[err] [EQ->Y] 

[err] [EQ->Y] y: 0 x: 0 Year: 710
[err] [EQ->Y] y: 0 x: 0 Year: 711
[err] [EQ->Y] y: 0 x: 0 Year: 712
[err] [EQ->Y] y: 0 x: 0 Year: 713
[err] [EQ->Y] y: 0 x: 0 Year: 714
[err] [EQ->Y] y: 0 x: 0 Year: 715
[err] [EQ->Y] y: 0 x: 0 Year: 716
[err] [EQ->Y] y: 0 x: 0 Year: 717
[err] [EQ->Y] y: 0 x: 0 Year: 718
[err] [EQ->Y] y: 0 x: 0 Year: 719
[err] [EQ->Y] y: 0 x: 0 Year: 720
[err] [EQ->Y] y: 0 x: 0 Year: 721
[err] [EQ->Y] y: 0 x: 0 Year: 722
[err] [EQ->Y] y: 0 x: 0 Year: 723
[err] [EQ->Y] y: 0 x: 0 Year: 724
[err] [EQ->Y] y: 0 x: 0 Year: 725
[err] [EQ->Y] y: 0 x: 0 Year: 726
[err] [EQ->Y] y: 0 x: 0 Year: 727
[err] [EQ->Y] y: 0 x: 0 Year: 728
[err] [EQ->Y] y: 0 x: 0 Year: 729
[err] [EQ->Y] y: 0 x: 0 Year: 730
[err] [EQ->Y] y: 0 x: 0 Year: 731
[err] [EQ->Y] y: 0 x: 0 Year: 732
[err] [EQ->Y] y: 0 x: 0 Year: 733
[err] [EQ->Y] y: 0 x: 0 Year: 734
[err] [EQ->Y] y: 0 x: 0 Year: 735
[err] [EQ->Y] y: 0 x: 0 Year: 736
[err] [EQ->Y] y: 0 x: 0 Year: 737
[err] [EQ->Y] y: 0 x: 0 Year: 738
[err] [EQ->Y] 

[err] [EQ->Y] y: 0 x: 0 Year: 951
[err] [EQ->Y] y: 0 x: 0 Year: 952
[err] [EQ->Y] y: 0 x: 0 Year: 953
[err] [EQ->Y] y: 0 x: 0 Year: 954
[err] [EQ->Y] y: 0 x: 0 Year: 955
[err] [EQ->Y] y: 0 x: 0 Year: 956
[err] [EQ->Y] y: 0 x: 0 Year: 957
[err] [EQ->Y] y: 0 x: 0 Year: 958
[err] [EQ->Y] y: 0 x: 0 Year: 959
[err] [EQ->Y] y: 0 x: 0 Year: 960
[err] [EQ->Y] y: 0 x: 0 Year: 961
[err] [EQ->Y] y: 0 x: 0 Year: 962
[err] [EQ->Y] y: 0 x: 0 Year: 963
[err] [EQ->Y] y: 0 x: 0 Year: 964
[err] [EQ->Y] y: 0 x: 0 Year: 965
[err] [EQ->Y] y: 0 x: 0 Year: 966
[err] [EQ->Y] y: 0 x: 0 Year: 967
[err] [EQ->Y] y: 0 x: 0 Year: 968
[err] [EQ->Y] y: 0 x: 0 Year: 969
[err] [EQ->Y] y: 0 x: 0 Year: 970
[err] [EQ->Y] y: 0 x: 0 Year: 971
[err] [EQ->Y] y: 0 x: 0 Year: 972
[err] [EQ->Y] y: 0 x: 0 Year: 973
[err] [EQ->Y] y: 0 x: 0 Year: 974
[err] [EQ->Y] y: 0 x: 0 Year: 975
[err] [EQ->Y] y: 0 x: 0 Year: 976
[err] [EQ->Y] y: 0 x: 0 Year: 977
[err] [EQ->Y] y: 0 x: 0 Year: 978
[err] [EQ->Y] y: 0 x: 0 Year: 979
[err] [EQ->Y] 

[err] [SP->Y] y: 0 x: 0 Year: 190
[err] [SP->Y] y: 0 x: 0 Year: 191
[err] [SP->Y] y: 0 x: 0 Year: 192
[err] [SP->Y] y: 0 x: 0 Year: 193
[err] [SP->Y] y: 0 x: 0 Year: 194
[err] [SP->Y] y: 0 x: 0 Year: 195
[err] [SP->Y] y: 0 x: 0 Year: 196
[err] [SP->Y] y: 0 x: 0 Year: 197
[err] [SP->Y] y: 0 x: 0 Year: 198
[err] [SP->Y] y: 0 x: 0 Year: 199
[err] [SP->Y] y: 0 x: 0 Year: 200
[err] [SP->Y] y: 0 x: 0 Year: 201
[err] [SP->Y] y: 0 x: 0 Year: 202
[err] [SP->Y] y: 0 x: 0 Year: 203
[err] [SP->Y] y: 0 x: 0 Year: 204
[err] [SP->Y] y: 0 x: 0 Year: 205
[err] [SP->Y] y: 0 x: 0 Year: 206
[err] [SP->Y] y: 0 x: 0 Year: 207
[err] [SP->Y] y: 0 x: 0 Year: 208
[err] [SP->Y] y: 0 x: 0 Year: 209
[err] [SP->Y] y: 0 x: 0 Year: 210
[err] [SP->Y] y: 0 x: 0 Year: 211
[err] [SP->Y] y: 0 x: 0 Year: 212
[err] [SP->Y] y: 0 x: 0 Year: 213
[err] [SP->Y] y: 0 x: 0 Year: 214
[err] [SP->Y] y: 0 x: 0 Year: 215
[err] [SP->Y] y: 0 x: 0 Year: 216
[err] [SP->Y] y: 0 x: 0 Year: 217
[err] [SP->Y] y: 0 x: 0 Year: 218
[err] [SP->Y] 

In [27]:
!ls /data/workflows/BONA-black-spruce/output/

ALD_yearly_eq.nc	    MINEC_yearly_eq.nc
ALD_yearly_sp.nc	    MINEC_yearly_sp.nc
ALD_yearly_tr.nc	    MINEC_yearly_tr.nc
AVLN_yearly_eq.nc	    NPP_monthly_eq.nc
AVLN_yearly_sp.nc	    NPP_monthly_sp.nc
AVLN_yearly_tr.nc	    NPP_monthly_tr.nc
BURNVEG2AIRC_monthly_eq.nc  ORGN_yearly_eq.nc
BURNVEG2AIRC_monthly_sp.nc  ORGN_yearly_sp.nc
BURNVEG2AIRC_monthly_tr.nc  ORGN_yearly_tr.nc
CMTNUM_yearly_eq.nc	    RG_monthly_eq.nc
CMTNUM_yearly_sp.nc	    RG_monthly_sp.nc
CMTNUM_yearly_tr.nc	    RG_monthly_tr.nc
DEEPC_yearly_eq.nc	    RH_monthly_eq.nc
DEEPC_yearly_sp.nc	    RH_monthly_sp.nc
DEEPC_yearly_tr.nc	    RH_monthly_tr.nc
EET_monthly_eq.nc	    RM_monthly_eq.nc
EET_monthly_sp.nc	    RM_monthly_sp.nc
EET_monthly_tr.nc	    RM_monthly_tr.nc
GPP_monthly_eq.nc	    SHLWC_monthly_eq.nc
GPP_monthly_sp.nc	    SHLWC_monthly_sp.nc
GPP_monthly_tr.nc	    SHLWC_monthly_tr.nc
LAI_monthly_eq.nc	    TLAYER_monthly_eq.nc
LAI_monthly_sp.nc	    TLAYER_monthly_sp.nc
LAI_monthly_tr.nc	    TLAYER_monthly_tr.nc
LAYERDE